#  **Fine Tuning LLM by - Dirct Performance Optimization(DPO) method**

- This script ran in **Colab with GPU(T4) Enabled**


- Here **TinyLlama/TinyLlama-1.1B-Chat-v1.0** model used.
  - https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0
- This script having some issue. Step2 DPO didnt complete properly


- 1st we will do SFT then do DOP on that
- SFT and DPO both are FT. Here  Supervised Fine tuning(SFT) done 1st and then the Direct performance Optimization(DPO). Here DPO are done more on chatbot related requirement where conversation type model. DPO something like RLHF
- We can also prepare finetuning data for DPO, refer this link or use existing data which we used in below code.
  - https://www.kaggle.com/code/aisuko/supervised-fine-tuning-llama2-with-dpo


### **Steps Followed**
####  **STEP 1: Fine Tuning LLM by - SFT LORA METHOD**
- Connect to hugging face to download model
- List all the Fine tuning model param/config
- Use sft_config parameter
- Main class to Fine tune the model
  - Create Train SFT class
  - Set LoraConfig
- **Download tatsu-lab/alpaca data from hugging face**
  - https://huggingface.co/datasets/tatsu-lab/alpaca
  - Alpaca is a dataset of 52,000 instructions and demonstrations generated by OpenAI's text-davinci-003 engine. This instruction data can be used to conduct instruction-tuning for language models and make the language model follow instruction better.
- **Execute Fine tune train method**
  - In this step it ask for weight and bias login api key and saves fine tuned model there
      ![plot](./image/ask_wandb_login.png)
  - Syncing run sft-tiny-chatbot to Weights & Biases (docs)
      ![plot](./image/training_log_in_wandb.png)
  - View project at https://wandb.ai/prabhavathi-melady-prudential-singapore/huggingface
  - View run at https://wandb.ai/prabhavathi-melady-prudential-singapore/huggingface/runs/12gwjefi

    - trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044
    - This means only *2,252,800* parameter are yused for training.
  - Same **PrabhaB/sft-tiny-chatbot** model saved in hugging face also, its SFT (Supervised Fine tuned ) Model


####  **STEP 2:  Preference Alignmemnt- DPO**
- DOP is kind of RLHF Training, it should be done after SFT(Supervised Fine tuning)
- List all the DPO evaluated  param/config

- Main class to do DPO
  - Here copied same old structure used in above SFT
- **Download "Anthropic/hh-rlhf data from hugging face**
- DOP is like retraining process, so download RLHF data again
  - https://huggingface.co/datasets/Anthropic/hh-rlhf
- This repository provides access to two different kinds of data:
  - Human preference data about helpfulness and harmlessness
  - Human-generated and annotated red teaming dialogues

### **Inferencing on DPO model**
- In this step retrive the fine tuned model from hugging face
- Create pipeline for inference
- Inference by newly fine tuned model

## **After FineTune model saved in local Colab and Hugging face Model**

  ![plot](./image/fine_tune_model.png)
  ![plot](./image/fine_tune_model_1.png)
  ![plot](./image/model_saved_in_hf.png)


## **Lora_rank_decomposition**

  ![plot](./image/Lora_rank_decomposition.png)

## **dpo_model_saved**

  ![plot](./image/dpo_model_saved.png)



#  **STEP 1: Fine Tuning LLM by - SFT LORA METHOD**

## Install Libraries

## Error:
- AttributeError: 'TrainingArguments' object has no attribute 'model_init_kwargs'
- To resolve it instaled below library version

!pip install “torch==2.1.2” tensorboard
!pip install --upgrade
“transformers==4.38.2”
“datasets==2.16.1”
“accelerate==0.26.1”
“evaluate==0.4.1”
“bitsandbytes==0.42.0”
“trl==0.7.11”
“peft==0.8.2”

In [ ]:
!pip show transformers

Name: transformers
Version: 4.44.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [ ]:
# !pip list

In [ ]:
!pip install trl peft accelerate datasets bitsandbytes -q #trl for fine tune model - Transformer Reinforcement learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━ 113.0/122.4 MB 121.9 MB/s eta 0:00:01ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━ 114.0/122.4 MB 112.5 MB/s eta 0:00:01


In [14]:
# !pip install torch==2.1.2 tensorboard -q


In [15]:
# !pip install --upgrade transformers==4.38.2 datasets==2.16.1 accelerate==0.26.1 evaluate==0.4.1 bitsandbytes==0.43.2 trl==0.7.11 peft==0.9.0 -q

## Connect to hugging face to download model

In [ ]:
from huggingface_hub import notebook_login
notebook_login()  # Use Hugging face write token

In [ ]:
from transformers import AutoModelForCausalLM
import torch
from datasets import load_dataset,Dataset
from trl import SFTTrainer #Supervised finetuning trainer
from accelerate import Accelerator
import os
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments

## List all the Fine tuning model param/config
- Use sft_config parameter

In [ ]:
sft_config={

            "model_ckpt": "TinyLlama/TinyLlama-1.1B-Chat-v1.0", # Model checkpoint
            "load_in_4bit": True, #For quantization
            "device_map": {"": Accelerator().local_process_index},  #Its fir gpu
            "torch_dtype": torch.float16,
            "trust_remote_code": True,

            #LoraConfig
            "use_lora": True,
            "r": 16, #Rank of the lora
            "lora_alpha": 16,
            "lora_dropout": 0.05,
            "bias": "none",
            "task_type": "CAUSAL_LM",
            "target_modules": ["q_proj", "v_proj"],

            #TrainingArguments
            "output_dir": "sft-tiny-chatbot",
            "per_device_train_batch_size": 1,
            "gradient_accumulation_steps": 1,
            "optim": "paged_adamw_32bit", #Which optimizer used
            "learning_rate": 2e-4,
            "lr_scheduler_type": "cosine",
            "save_strategy": "epoch",
            "logging_steps": 100,
            "num_train_epochs": 1,
            "max_steps": 250,
            "fp16": True,
            "push_to_hub": True,
            "train_cln_name": "text",
            "packing": False,
            "max_seq_length": 512,
            "neftune_noise_alpha": 5

}

## Main class to Fine tune the model
- Create Train SFT class
- Set LoraConfig

In [ ]:
class TrainSFT:

  def __init__(self,data,config):
    self.data=data      #Data used for finetuning
    self.config=config  #This is sft_config only, we pass all sft config while calling TrainSFT class

  def prepare_lora_model(self):
    print("\t\t\tPREPARE MODEL FOR FINETUNING")

    #Set LoraConfig
    self.lora_config= LoraConfig(r=self.config["r"],  #rank
                                    lora_alpha=self.config["lora_alpha"],
                                    lora_dropout=self.config["lora_dropout"],
                                    bias=self.config["bias"],
                                    task_type=self.config["task_type"],
                                    target_modules=self.config["target_modules"])

    self.model= get_peft_model(self.model,self.lora_config)

  #Load the model which is going to be finetuned and Model's tokenizer from HF
  def load_model_tokenizer(self):
    self.model = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"], # This is the model name used for finetune: "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
                            load_in_4bit=self.config["load_in_4bit"],#Define quantization method
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"])
    print("\n====================================================================\n")
    print("\t\t\tDOWNLOADED MODEL")
    print(model)
    print("\n====================================================================\n")

    self.model.config.use_cache=False
    self.model.config.pretraining_tp=1
    self.model = prepare_model_for_kbit_training(self.model)

    print("\n====================================================================\n")
    print("\t\t\tMODEL CONFIG UPDATED")
    print("\n====================================================================\n")

    if self.config["use_lora"]:
      self.prepare_lora_model()

    print("\n====================================================================\n")
    print("\t\t\tPREPARED MODEL FOR FINETUNING")
    print(model)
    print("\n====================================================================\n")

    # Load this "TinyLlama/TinyLlama-1.1B-Chat-v1.0"s base tokenizer model from HF
    self.tokenizer = AutoTokenizer.from_pretrained(self.config["model_ckpt"])
    self.tokenizer.pad_token = self.tokenizer.eos_token

  def set_training_args(self):


    print("\n====================================================================\n")
    print("\t\t\tSET TRAINING ARGUMENTS")
    print(model)
    print("\n====================================================================\n")

    return TrainingArguments(
                                    output_dir=self.config["output_dir"],
                                    per_device_train_batch_size=self.config["per_device_train_batch_size"],
                                    gradient_accumulation_steps=self.config["gradient_accumulation_steps"],
                                    optim=self.config["optim"],
                                    learning_rate=self.config["learning_rate"],
                                    lr_scheduler_type=self.config["lr_scheduler_type"],
                                    save_strategy=self.config["save_strategy"],
                                    logging_steps=self.config["logging_steps"],
                                    num_train_epochs=self.config["num_train_epochs"],
                                    max_steps=self.config["max_steps"],
                                    fp16=self.config["fp16"],
                                    push_to_hub=self.config["push_to_hub"],  #Push to hub set to True
                                    neftune_noise_alpha=self.config["neftune_noise_alpha"]
                                )
  def create_trainer(self):

        self.load_model_tokenizer()  #call func where model and model tokenizer downloaded from HF
        if self.config["use_lora"]: # If use_lora set to True
            print(self.model.print_trainable_parameters())
            self.trainer = SFTTrainer( #SFTTrainer = Supervised finetuning trainer class imported from trl
                                    model=self.model,
                                    train_dataset=self.data,
                                    peft_config=self.lora_config,
                                    dataset_text_field=self.config["train_cln_name"],
                                    args=self.set_training_args(),
                                    tokenizer=self.tokenizer,
                                    packing=self.config["packing"],
                                    max_seq_length=self.config["max_seq_length"]
                                )
        else:  # Dont know what is difference in if and else content
            self.trainer = SFTTrainer(
                                    model=self.model,
                                    train_dataset=self.data,
                                    dataset_text_field=self.config["train_cln_name"],
                                    args=self.set_training_args(),
                                    tokenizer=self.tokenizer,
                                    packing=self.config["packing"],
                                    max_seq_length=self.config["max_seq_length"]
                                )

  def train_and_save_model(self):
    self.create_trainer()
    self.trainer.train()
    self.trainer.save_model(self.config["output_dir"]) # save the trained model in local  output_dir =  "sft-tiny-chatbot"
    self.tokenizer.save_pretrained(self.config["output_dir"])  # save the trained model tokenizer in local  output_dir =  "sft-tiny-chatbot"

    #Push the model to HF not required in this way,its already set in training argument: push_to_hub=self.config["push_to_hub"]
    #trainer.push_to_hub()



### **Download tatsu-lab/alpaca data from hugging face**
- https://huggingface.co/datasets/tatsu-lab/alpaca
- Alpaca is a dataset of 52,000 instructions and demonstrations generated by OpenAI's text-davinci-003 engine. This instruction data can be used to conduct instruction-tuning for language models and make the language model follow instruction better.

In [ ]:
import torch
from datasets import load_dataset,Dataset

In [ ]:
def create_data():
    data = load_dataset("tatsu-lab/alpaca", split="train")
    data_df = data.to_pandas()
    data_df = data_df[:700]
    data_df["text"] = data_df[["input", "instruction", "output"]].apply(lambda x: "Human: " + x["instruction"] + " " + x["input"] + " Assistant: "+ x["output"], axis=1)
    data = Dataset.from_pandas(data_df)
    return data

data = create_data()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### Execute Fine tune train method
- In this step it ask for weight and bias login api key and saves fine tuned model there
  - Syncing run sft-tiny-chatbot to Weights & Biases (docs)
  - View project at https://wandb.ai/prabhavathi-melady-prudential-singapore/huggingface
  - View run at https://wandb.ai/prabhavathi-melady-prudential-singapore/huggingface/runs/12gwjefi

  - trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044
    - This means only *2,252,800* parameter are yused for training.

In [ ]:
train_sft = TrainSFT(data, sft_config)
train_sft.train_and_save_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.




			DOWNLOADED MODEL


NameError: name 'model' is not defined

#  **STEP 2:  Preference Alignmemnt- DPO**
- DOP is kind of RLHF Training, it should be done after SFT(Supervised Fine tuning)


## Install Libraries

In [ ]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import DPOTrainer
import pandas as pd
from accelerate import Accelerator
from peft import get_peft_model, LoraConfig

## List all the DPO evaluated  param/config


In [ ]:
dpo_config = {
            #snshrivas10/sft-tiny-chatbot : This model is a fine-tuned version of TinyLlama/TinyLlama-1.1B-Chat-v1.0 on the dataset by sunny savita
            "model_ckpt": "PrabhaB/sft-tiny-chatbot",
            "load_in_4bit": True,
            "device_map": {"": Accelerator().local_process_index},  #Its for gpu
            "torch_dtype": torch.float16,
            "trust_remote_code": True,

            #LoraConfig
            "use_lora": True,
            "r": 8,
            "lora_alpha": 8,
            "lora_dropout": 0.05,
            "bias": "none",
            "task_type": "CAUSAL_LM",
            "target_modules": ["q_proj", "v_proj"],

            #TrainingArguments
            "output_dir": "tiny-chatbot-dpo",
            "per_device_train_batch_size": 1,
            "gradient_accumulation_steps": 1,
            "optim": "paged_adamw_32bit", #Which optimizer used
            "learning_rate": 2e-4,
            "lr_scheduler_type": "cosine",
            "save_strategy": "epoch",
            "logging_steps": 100,
            "num_train_epochs": 1,
            "max_steps": 250,
            "fp16": True,
            "push_to_hub": True,
            "train_cln_name": "text",
            "packing": False,
            "neftune_noise_alpha": 5,
            "beta": 0.1,
            "loss_type": "kto_pair",
            "max_length": 768,
            "max_prompt_length": 512,
            "max_target_length": 256,
            "is_encoder_decoder": False
        }

## Main class to do DPO
- Here copied same old structure used in above SFT

In [ ]:
class TrainDPO:

  def __init__(self,data,config):
    self.data=data  #PASS data to do DPO
    self.config=config

  def prepare_lora_model(self):
        print("\t\t\tPREPARE MODEL FOR FINETUNING")

        #Set LoraConfig
        self.lora_config = LoraConfig(
                                    r=self.config["r"],
                                    lora_alpha=self.config["lora_alpha"],
                                    lora_dropout=self.config["lora_dropout"],
                                    bias=self.config["bias"],
                                    task_type=self.config["task_type"],
                                    target_modules=self.config["target_modules"]
                                )
        self.model = get_peft_model(self.model, self.lora_config)
        self.model_ref = get_peft_model(self.model_ref, self.lora_config)

  #Load the model which is going to be finetuned and Model's tokenizer from HF
  def load_model_tokenizer(self):

        self.model = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"]
                        )

        # Dont know why 2 time did same for model and model_ref
        self.model_ref = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"]
                        )

        print("\n====================================================================\n")
        print("\t\t\tDOWNLOADED MODEL")
        print(self.model)
        print("\n====================================================================\n")

        self.model.config.use_cache=False
        self.model.config.pretraining_tp=1
        self.model = prepare_model_for_kbit_training(self.model)

        print("\n====================================================================\n")
        print("\t\t\tMODEL CONFIG UPDATED")
        print("\n====================================================================\n")

        if self.config["use_lora"]:
            self.prepare_lora_model()

        print("\n====================================================================\n")
        print("\t\t\tPREPARED MODEL FOR FINETUNING")
        print(self.model)
        print("\n====================================================================\n")

        # Load this "snshrivas10/sft-tiny-chatbot"s base tokenizer model from HF
        self.tokenizer = AutoTokenizer.from_pretrained(self.config["model_ckpt"])
        self.tokenizer.pad_token = self.tokenizer.eos_token

  def set_training_args(self):

        print("\n====================================================================\n")
        print("\t\t\tSET TRAINING ARGUMENTS")
        # print(self.model)
        print("\n====================================================================\n")

        return TrainingArguments(
                                    output_dir=self.config["output_dir"],
                                    per_device_train_batch_size=self.config["per_device_train_batch_size"],
                                    gradient_accumulation_steps=self.config["gradient_accumulation_steps"],
                                    optim=self.config["optim"],
                                    learning_rate=self.config["learning_rate"],
                                    lr_scheduler_type=self.config["lr_scheduler_type"],
                                    save_strategy=self.config["save_strategy"],
                                    logging_steps=self.config["logging_steps"],
                                    num_train_epochs=self.config["num_train_epochs"],
                                    max_steps=self.config["max_steps"],
                                    fp16=self.config["fp16"],
                                    push_to_hub=self.config["push_to_hub"],
                                    neftune_noise_alpha=self.config["neftune_noise_alpha"]
                                )


  def create_trainer(self):

        self.load_model_tokenizer() #call func where model and model tokenizer downloaded from HF
        if self.config["use_lora"]:
            print(self.model.print_trainable_parameters())
            self.trainer = DPOTrainer(
                                        self.model,
                                        self.model_ref,  # Only in dpo, model_ref comes during DPOTrainer
                                        args=self.set_training_args(),
                                        train_dataset=self.data,
                                        tokenizer=self.tokenizer,
                                        beta=self.config["beta"],
                                        loss_type=self.config["loss_type"],
                                        max_length=self.config["max_length"],
                                        max_prompt_length=self.config["max_prompt_length"],
                                        max_target_length=self.config["max_target_length"],
                                        is_encoder_decoder=self.config["is_encoder_decoder"]
                                    )
        else: # Dont know what is difference in if and else content
            self.trainer = DPOTrainer(
                                        self.model,
                                        self.model_ref,   # Only in dpo, model_ref comes during DPOTrainer
                                        peft_config=self.lora_config,
                                        args=self.set_training_args(),
                                        train_dataset=self.data,
                                        tokenizer=self.tokenizer,
                                        beta=self.config["beta"],
                                        loss_type=self.config["loss_type"],
                                        max_length=self.config["max_length"],
                                        max_prompt_length=self.config["max_prompt_length"],
                                        max_target_length=self.config["max_target_length"],
                                        is_encoder_decoder=self.config["is_encoder_decoder"]
                                    )
  def train_and_save_model(self):
    self.create_trainer()
    self.trainer.train()
    self.trainer.save_model(self.config["output_dir"])  # save the trained model in local  output_dir =  "tiny-chatbot-dpo"
    self.tokenizer.save_pretrained(self.config["output_dir"])


### **Download "Anthropic/hh-rlhf data from hugging face**
- DOP is like retraining process, so download RLHF data again
- https://huggingface.co/datasets/Anthropic/hh-rlhf
- This repository provides access to two different kinds of data:
  - Human preference data about helpfulness and harmlessness
  - Human-generated and annotated red teaming dialogues

In [ ]:
def create_data():
    df = load_dataset("Anthropic/hh-rlhf", split="train").to_pandas()
    df["prompt"] = df["chosen"].apply(lambda x: x.split("Assistant: ")[0])
    df["chosen"] = df["chosen"].apply(lambda x: "Assistant: "+ x.split("Assistant: ")[-1])
    df["rejected"] = df["rejected"].apply(lambda x: "Assistant: " + x.split("Assistant: ")[-1])
    df = df.sample(1000)
    data = Dataset.from_pandas(df)
    return data
data = create_data()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
train_dpo = TrainDPO(data, dpo_config)
train_dpo.train_and_save_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

TypeError: LoraConfig.__init__() got an unexpected keyword argument 'layer_replication'

## **ISSUE**
- It fails in above training step with below error.
 'TrainingArguments' object has no attribute 'model_init_kwargs'
 - So stoped here
 - If its successfull it should save this dpo trained model in local and hugging face with name - **tiny-chatbot-dpo**

# Inferencing on DPO model

In [ ]:
from peft import AutoPeftModelForCausalLM

import torch
from transformers import pipeline

In [ ]:
model=AutoPeftModelForCausalLM.from_pretrained("snshrivas10/tiny-chatbot-dpo")  # My dpo model part failed, so suing sunny's dpo ft model for inference
model

adapter_config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_

## Create Pipeline

In [ ]:
pipe=pipeline("text-generation",
              model=model,
              tokenizer="snshrivas10/tiny-chatbot-dpo",
              torch_dtype=torch.bfloat16,
              device_map="auto") # use same model name as tokenizer

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCa

### Example 1

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]

In [ ]:
#Do apply_chat_template  on prompt
prompt=pipe.tokenizer.apply_chat_template(messages,tokenize=False,add_generation_prompt=True)
prompt

'<|system|>\nYou are a friendly chatbot who always responds in the style of a pirate</s>\n<|user|>\nHow many helicopters can a human eat in one sitting?</s>\n<|assistant|>\n'

In [ ]:
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
outputs[0]["generated_text"]

'<|system|>\nYou are a friendly chatbot who always responds in the style of a pirate</s>\n<|user|>\nHow many helicopters can a human eat in one sitting?</s>\n<|assistant|>\nWhile there is no scientific evidence supporting this claim, it is widely believed that humans can eat up to 10-12 helicopters in one sitting. The exact number of helicopters that a human can eat depends on their size, how hungry they are, and the specific type of helicopter. However, it is believed that humans can consume up to 10-12 helicopters as a quick snack or even as a meal.'

### Example 2

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "Discuss the causes of the Great Depression"},
]

In [ ]:
#Do apply_chat_template  on prompt
prompt=pipe.tokenizer.apply_chat_template(messages,tokenize=False,add_generation_prompt=True)
prompt

'<|system|>\nYou are a friendly chatbot who always responds in the style of a pirate</s>\n<|user|>\nDiscuss the causes of the Great Depression</s>\n<|assistant|>\n'

In [ ]:
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
outputs[0]["generated_text"]

'<|system|>\nYou are a friendly chatbot who always responds in the style of a pirate</s>\n<|user|>\nDiscuss the causes of the Great Depression</s>\n<|assistant|>\nThe Great Depression, also known as the Great Recession, was a severe economic depression that occurred in the United States between 1929 and 1939. It was triggered by several factors, including:\n\n1. Stock Market Crash: In October 1929, the New York Stock Exchange crashed, leading to a massive decline in stock prices. This caused the wealth of many Americans to decline significantly, and they were unable to sell their assets to pay their bills.\n\n2. Divergence in Economic Growth: The United States had been growing rapidly during the early 20th century, with significant progress in technology, education, and innovation. However, in the 1920s, this growth slowed down, and the country experienced stagnant wages and unemployment.\n\n3. International Conflicts: The United States was involved in several world wars, including Wor

In [ ]:
outputs

[{'generated_text': '<|system|>\nYou are a friendly chatbot who always responds in the style of a pirate</s>\n<|user|>\nDiscuss the causes of the Great Depression</s>\n<|assistant|>\nThe Great Depression, also known as the Great Recession, was a severe economic depression that occurred in the United States between 1929 and 1939. It was triggered by several factors, including:\n\n1. Stock Market Crash: In October 1929, the New York Stock Exchange crashed, leading to a massive decline in stock prices. This caused the wealth of many Americans to decline significantly, and they were unable to sell their assets to pay their bills.\n\n2. Divergence in Economic Growth: The United States had been growing rapidly during the early 20th century, with significant progress in technology, education, and innovation. However, in the 1920s, this growth slowed down, and the country experienced stagnant wages and unemployment.\n\n3. International Conflicts: The United States was involved in several world